In [ ]:
import wave
import os
import librosa
import soundfile as sf
from pydub import AudioSegment
import sndhdr

In [9]:
windows_path = "D:/Desktop/coqui_tts/"
root_path = "/home/pnx/Desktop/paddle/"
path = windows_path
file_generation = path + "output/" + "2025-03-31-15_52_01.wav"
file_reference = path + "data/" + "2.wav"

In [18]:
def get_sampling_rate_wav(file_path):
    print(f"Processing file: {file_path}")
    
    if file_path.lower().endswith('.wav'):
        # 使用 sndhdr 获取 WAV 文件的信息
        sndheader = sndhdr.what(file_path)
        print(f"sndhdr output: {sndheader}")
        return sndheader.filetype, sndheader.framerate, sndheader.nchannels
    else:
        try:
            audio = AudioSegment.from_file(file_path)
            # 获取采样率和通道数
            sample_rate = audio.frame_rate
            num_channels = audio.channels
            print(f"Sample rate: {sample_rate}, Channels: {num_channels}")
            
            output_path = file_path.replace(os.path.splitext(file_path)[1], ".wav")
            audio.export(output_path, format="wav")
            print(f"Converted to WAV: {output_path}")
            
            return 'converted_wav', sample_rate, num_channels
        
        except Exception as e:
            print(f"Error while processing the file: {e}")
            return None, None, None


def resample_audio(input_path, output_path, target_sr=16000):
    # 加载音频文件，sr=None 表示加载时不进行采样率重采样
    y, sr = librosa.load(input_path, sr=None, mono=False)
    num_channels = y.shape[0]

    if sr != target_sr:
        y_resampled = librosa.resample(y, orig_sr=sr, target_sr=target_sr)

        sf.write(output_path, y_resampled, target_sr)
        print(f"音频 {input_path} 已重采样为 {target_sr}Hz，并保存为 {output_path}")
    else:
        # 如果采样率已经是目标采样率，直接保存
        sf.write(output_path, y, sr)
        print(f"音频 {input_path} 已是目标采样率 {target_sr}Hz，无需转换, save to {output_path}")

### 获取采样率数据

In [22]:
filetype, sampling_rate, channels = get_sampling_rate_wav(path+"data/2.mp3")
print(f"Sampling rate: {sampling_rate} Hz, Channels: {channels} ")

Processing file: D:/Desktop/coqui_tts/data/2.mp3
Sample rate: 48000, Channels: 2
Error while processing the file: name 'os' is not defined
Sampling rate: None Hz, Channels: None 


In [5]:
sndheader = sndhdr.what(file_generation)
print(sndheader)

SndHeaders(filetype='wav', framerate=22050, nchannels=1, nframes=1144656, sampwidth=16)


In [14]:
sample_generation = file_generation.replace("output/", "output/resample/")
sample_reference = file_reference.replace("data/", "output/resample/")
resample_audio(file_generation, sample_generation)
resample_audio(file_reference, sample_reference)

音频 /home/pnx/Desktop/paddle/output/2025-03-31-15_52_01.wav 已重采样为 16000Hz，并保存为 /home/pnx/Desktop/paddle/output/resample/2025-03-31-15_52_01.wav
音频 /home/pnx/Desktop/paddle/data/2.wav 已是目标采样率 16000Hz，无需转换


In [26]:
print(sample_reference)

D:/Desktop/coqui_tts/output/resample/2.wav
